In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from datetime import timedelta
from dateutil import parser
import re
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os

C:\Users\brgoh\anaconda33\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\brgoh\anaconda33\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\brgoh\anaconda33\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\brgoh\anaconda33\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: P

In [2]:
df = pd.read_csv('tweets_stocks_combined_5mins.csv')

In [4]:
df

,id,cleaned_text,favorites,retweets,date,tweet_datetime,date_part,time_part,hour,year,...,entity_type,datetime_5mins_after,price_5mins_after,eod_price,5mins_price_diff_abs,5mins_price_diff_perc,eod_price_diff_abs,eod_price_diff_perc,5mins_price_diff_perc_magnitude,eod_price_diff_perc_magnitude
0,9.353400e+17,thank you rand!,42793,9125,2017-11-28 02:50:00,2017-11-28 10:50:00,2017-11-28,10:50:00,10,2017,...,['ORG'],2017-11-28 10:55:00,261.085000,262.87,-0.015000,-0.000057,1.770000,0.006779,0.000057,0.006779
1,8.997980e+17,"join me live from fort myer in arlington, virg...",36009,4891,2017-08-22 01:00:00,2017-08-22 09:00:00,2017-08-22,09:00:00,9,2017,...,"['GPE', 'GPE', 'GPE', 'ORG']",2017-08-22 09:05:00,243.670000,245.44,0.000000,0.000000,1.770000,0.007264,0.000000,0.007264
2,8.939700e+17,thank you nicole!,43367,8275,2017-05-08 23:01:00,2017-05-09 07:01:00,2017-05-09,07:01:00,7,2017,...,['PERSON'],2017-05-09 07:06:00,239.920000,239.44,0.045000,0.000188,-0.435000,-0.001813,0.000188,0.001813
3,8.819770e+17,thank you to shawn steel for the nice words on .,50956,7465,2017-03-07 20:44:00,2017-03-08 04:44:00,2017-03-08,04:44:00,4,2017,...,['PERSON'],2017-03-08 04:49:00,236.913333,236.56,0.033333,0.000141,-0.320000,-0.001351,0.000141,0.001351
4,8.778460e+17,great night in iowa - special people. thank you!,56446,8039,2017-06-22 11:11:00,2017-06-22 19:11:00,2017-06-22,19:11:00,19,2017,...,"['TIME', 'GPE']",2017-06-22 19:16:00,242.905000,242.84,0.025000,0.000103,-0.040000,-0.000165,0.000103,0.000165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2664,9.990960e+17,if the person placed very early into my campai...,78529,20098,2018-05-23 01:13:00,2018-05-23 09:13:00,2018-05-23,09:13:00,9,2018,...,"['ORG', 'GPE', 'PERSON', 'PERSON']",2018-05-23 09:18:00,271.100000,273.36,0.060000,0.000221,2.320000,0.008560,0.000221,0.008560
2665,9.874600e+17,so general michael flynn’s life can be totally...,93569,25259,2018-04-20 10:34:00,2018-04-20 18:34:00,2018-04-20,18:34:00,18,2018,...,"['PERSON', 'PERSON', 'PERSON', 'ORDINAL', 'GPE...",2018-04-20 18:39:00,266.866667,266.61,0.046667,0.000175,-0.210000,-0.000787,0.000175,0.000787
2666,9.870960e+17,"my thoughts, prayers and condolences are with ...",62645,16081,2018-04-19 22:30:00,2018-04-20 06:30:00,2018-04-20,06:30:00,6,2018,...,"['CARDINAL', 'ORG', 'ORG', 'DATE', 'PERSON', '...",2018-04-20 06:35:00,268.715000,266.61,0.095000,0.000354,-2.010000,-0.007483,0.000354,0.007483
2667,9.863570e+17,today’s court decision means that congress mus...,56749,12426,2018-04-17 21:34:00,2018-04-18 05:34:00,2018-04-18,05:34:00,5,2018,...,"['DATE', 'ORG', 'ORG', 'ORG', 'ORG', 'ORG']",2018-04-18 05:39:00,270.600000,270.39,0.000000,0.000000,-0.210000,-0.000776,0.000000,0.000776


In [3]:
X = df.loc[:, 'cleaned_text2']
y = df.loc[:, '5mins_price_diff_perc']

KeyError: 'cleaned_text2'

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42

In [ ]:
for i in X_train:
    corpus_list.append(i.split(' '))

In [ ]:
word2vec_model = Word2Vec(corpus_list, min_count=1, size=100)
pretrained_weights = word2vec_model.wv.vectors

In [ ]:
num_words = [len(i) for i in corpus_list]
longest_sentence_len = max(num_words)

In [5]:
def sentence_to_indices_padded(sentences, longest_sentence_len):
    result = []
    for sentence in sentences:
        indices = []
        sentence_splitted = sentence.split()
        for word in sentence_splitted:
            if word in word2vec_model.wv.vocab:
                indices.append(word2vec_model.wv.vocab[word].index)
        result.append(indices)
    return keras.preprocessing.sequence.pad_sequences(result, maxlen=longest_sentence_len, padding='post')

In [ ]:
X_train_padded = sentence_to_indices_padded(X_train, longest_sentence_len)
X_test_padded = sentence_to_indices_padded(X_test, longest_sentence_len)

In [ ]:
embeddings_index = {}
f = open('glove/glove.twitter.27B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
tokenizer = keras.preprocessing.text.Tokenizer(nb_words=None)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
glove_word_count = dict(tokenizer.word_counts)
glove_sorted=dict(sorted(glove_word_count.items(), key=lambda x: x[1],reverse=True))
print(glove_sorted)

In [ ]:
w2c = dict()
for item in word2vec_model.wv.vocab:
    w2c[item]=word2vec_model.wv.vocab[item].count
w2c_sorted=dict(sorted(w2c.items(), key=lambda x: x[1],reverse=True))
print(w2c_sorted)

In [ ]:
glove_word_count = dict(tokenizer.word_counts)
glove_sorted=dict(sorted(glove_word_count.items(), key=lambda x: x[1],reverse=True))
print(glove_sorted)

In [ ]:
words_w2v = list(w2c.keys())
words_gloves = list(glove_word_count.keys())
extra = []\
for word in words_w2v:
    if word not in words_gloves:
        extra.append(word)
print(extra)

In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, 100))
count = 0
skipped_words = []
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        count += 1
        skipped_words.append(word)
vocab_size_glove, embedding_size_glove = embedding_matrix.shape
print(vocab_size_glove)
print(embedding_size)
print(count)
print(skipped_words)
embedding_layer_glove = Embedding(len(word_index) + 1,
                            100,
                            weights=[embedding_matrix],
                            input_length=longest_sentence_len,
                            trainable=False)

In [ ]:
def create_LSTM_model(pretrained_weights, longest_sentence_len):
    vocab_size, embedding_size = pretrained_weights.shape
    model = keras.Sequential()
    model.add(layers.Input(shape=longest_sentence_len, dtype='int32'))
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_size, weights=[pretrained_weights], trainable=False))  
    model.add(layers.LSTM(4, return_sequences=True, name='LSTM1'))
    model.add(layers.Dropout(0.25,name='Dropout1'))
    model.add(layers.LSTM(4, return_sequences=False, name='LSTM2'))
    model.add(layers.Dropout(0.25,name='Dropout2'))
    model.add(layers.Dense(4,name='Dense',activation='sigmoid'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(1,activation='linear'))
    return model

In [ ]:
model = create_LSTM_model(pretrained_weights, longest_sentence_len)

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [ ]:
from datetime import datetime

now = datetime.now()
dt_string = now.strftime(\"%d%m%Y %Hh%Mm\")

checkpoint_filepath = f'./model_a_checkpoint/{dt_string}.h5'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    verbose = 1,
    save_best_only=True)

model.fit(X_train_padded, y_train, validation_split=0.2, epochs=50, callbacks=[model_checkpoint_callback])